In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import numpy as np
# import PIL

In [2]:
# import data
folder="./dataset"
# transform = transforms.Compose([transforms.ToTensor()])
# transform = transforms.Compose([transforms.Resize((224, 224))])
data_train_val=torchvision.datasets.FashionMNIST(root=folder,train=True,download=True,transform=None)
data_test=torchvision.datasets.FashionMNIST(root=folder,train=False,download=True,transform=None)

In [3]:
classes=data_test.classes

In [4]:
# import matplotlib.pyplot as plt
# figure = plt.figure(figsize = (8,8))
# cols, rows = 4, 4
# ids=torch.randint(len(data_test), size = (cols*rows,))
# for i in range (1, cols*rows + 1):
#     image, label = data_test[ids[i-1].item()]
#     figure.add_subplot(rows, cols, i)
#     plt.title(classes[label])
#     plt.axis('off')
#     plt.imshow(image.squeeze(), cmap='gray')
#     # plt.imshow(image.squeeze())
# plt.show()

In [5]:
# data_train_val.data[:2000].shape
data_test.data.shape
# type(data_train_val.data)
# data_test.targets.shape

torch.Size([10000, 28, 28])

### Data Preparation (10 points)
- (a) Data cleansing and transformation (5 points)

In [6]:
class dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets, trans=None):
        self.x = inputs
        self.y = targets
        self.trans=trans
        
        # # for loss function
        # self.pos_indices = np.flatnonzero(targets==1)
        # self.pos_index_map = {}
        # for i, idx in enumerate(self.pos_indices):
        #     self.pos_index_map[idx] = i

    def __len__(self):
        return self.x.size()[0]

    def __getitem__(self, idx):
        # self.mode = 'train' #TODO
        # if self.mode == 'train':
        #    idx2 = self.pos_index_map[idx] if idx in self.pos_indices else -1

        if self.trans == None:
            # return (idx2, self.x[idx], self.y[idx])
            # print("no trans")
            return ( self.x[idx], self.y[idx])
        else:
            # return (idx2, self.trans(self.x[idx]), self.y[idx]) 
            # print("trans")
            return ( self.trans(self.x[idx]), self.y[idx]) 

def ds_trans(ds_input,trans_flag):
    data_input=ds_input.data.clone().detach()
    labels_input=ds_input.targets
    if data_input.ndim <= 3:
        # train_transform = transforms.Compose([
        #     transforms.ToPILImage(),
        #     transforms.Grayscale(3),
        #     transforms.Resize((32, 32)), 
        #     transforms.ToTensor(),
        #      transforms.Normalize(0.5, 0.5)
        # ])
        # train_transform = transforms.Compose([
        #     transforms.Resize((224, 224))
        # ])
        transform_train_val=transforms.Resize((64, 64))
        transform_test=transforms.Resize((64, 64))
    elif data_input.ndim == 4:
        # Should not come here
        print("WRONG!")

    data_input = data_input/255.0
    if torch.isnan(data_input).any():
        print("have NaN or Inf")
    # data_input = data_input.clone().detach()
    # data_input = torch.tensor(data_input, dtype=torch.float32)
    data_input = data_input[:,None,:,:]
    print(data_input.shape)
    # flag_labels = torch.tensor(flag_labels) 
    # flag_labels = flag_labels[:,None]
    # print(flag_labels.shape)
    
    if trans_flag==1:
        flag_ds_new = dataset(data_input, labels_input, trans=transform_train_val)
    elif trans_flag==2:
        flag_ds_new = dataset(data_input, labels_input, trans=transform_test)
    else:
        flag_ds_new = dataset(data_input, labels_input, trans=None)
    return (flag_ds_new)
    

In [7]:
ds_processed_train_val=ds_trans(ds_input=data_train_val,trans_flag=1)
ds_processed_test=ds_trans(ds_input=data_test,trans_flag=2)
# ds_processed.x.shape

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [8]:
from torch.utils.data.sampler import SubsetRandomSampler
ds_processed_train_split, ds_processed_val_split = torch.utils.data.random_split(data_train_val, [50000, 10000]) 
ds_processed_train_sampler=SubsetRandomSampler(ds_processed_train_split.indices)
ds_processed_val_sampler=SubsetRandomSampler(ds_processed_val_split.indices)

batch_size=64
# train_loader = torch.utils.data.DataLoader(dataset=ds_processed_train,
#                             batch_size=batch_size,
#                             shuffle=True)
loader_train = torch.utils.data.DataLoader(ds_processed_train_val, batch_size=batch_size,sampler=ds_processed_train_sampler,
                                            num_workers=2)
loader_val = torch.utils.data.DataLoader(ds_processed_train_val, batch_size=batch_size,sampler=ds_processed_val_sampler,
                                            num_workers=2)
loader_test = torch.utils.data.DataLoader(ds_processed_test, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [9]:
for batch_idx, (inputs, targets) in enumerate(loader_train):
    break
print(inputs.shape)
print(targets.shape)

/home/percool/anaconda3/envs/fashion/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/percool/anaconda3/envs/fashion/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None t

torch.Size([64, 1, 64, 64])
torch.Size([64])


In [39]:
# transform images size 28,28 to 224,224

from torchvision.transforms.functional import InterpolationMode
# data_transform = transforms.Compose(
#     [transforms.ToPILImage(),
#     transforms.Resize((224, 224), interpolation=PIL.Image.NEAREST), 
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[.5], std=[.5])])
data_transform = transforms.Compose(
    [ 
    transforms.Resize((224, 224))])
# data_train_val=torchvision.datasets.FashionMNIST(root=folder,train=True,download=True,transform=data_transform)
# data_test=torchvision.datasets.FashionMNIST(root=folder,train=False,download=True,transform=data_transform)
# data_transform(data_test.data)

# reshape torch.Size([10000, 28, 28]) to torch.Size([10000,1, 28, 28])
# data_transform=transforms.Resize((224, 224))

# temp=data_transform(data_test.data)
# data_test.data=temp
# temp.shape

# temp=data_test.data
# transform(temp)
temp= data_train_val.data[:, None, :, :]
print(temp.shape)
# data_transform = transforms.Compose(
#     [ 
#     transforms.Resize((224, 224))])
data_transform = transforms.Compose(
    [ 
    transforms.ToPILImage()])
temp2=data_transform(temp[1])
type(temp2)
# data_train_val.data=temp2

# torch.utils.data.Dataset
temp3=torch.tensor(temp,dtype=torch.float32)
temp4=temp3/255
temp4.shape
type(data_train_val.targets)
ds=torch.utils.data.Dataset(temp4,data_train_val.targets,transform=data_transform)

torch.Size([60000, 1, 28, 28])


/tmp/ipykernel_317275/738004040.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  temp3=torch.tensor(temp,dtype=torch.float32)


TypeError: Dataset() takes no arguments

- (b) Data splitting (i.e., training, validation, and test splits) (5 points)

In [ ]:

set_train, set_val = torch.utils.data.random_split(data_train_val, [50000, 10000])
len(set_train.indices)
type(set_val)
batch_size=64
loader_train = torch.utils.data.DataLoader(set_train, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
loader_val = torch.utils.data.DataLoader(set_val, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
loader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [ ]:
data_train_val.data[1].shape


torch.Size([28, 28])

## Model

In [10]:
device ='cuda' if torch.cuda.is_available else 'cpu'
print(f'Using {device} device')
# model = torch.hub.load('pytorch/vision:v0.10.0', 'wide_resnet50_2', pretrained=True)
model=torchvision.models.vgg13()
# model

Using cuda device


In [11]:
# model.conv1=torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.features[0]=torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model=model.to(device)
model

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): 

In [12]:
# Define hyperparameters
learning_rate = 1e-3
epochs = 50

# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
def trainloop (dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)
        
        # Compute the error rate
        prediction = model(X)
        loss = loss_fn(prediction, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f'loss:{loss:>7f} [{current:>5d}/{size:>5d}]')
            
def evaluate (dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [14]:
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    trainloop(loader_train, model, loss_fn, optimizer)
    print("Validation Error");
    evaluate(loader_val, model, loss_fn)
    print("Test Error");
    evaluate(loader_test, model, loss_fn)

Epoch 1
-------------------------------


/home/percool/anaconda3/envs/fashion/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/percool/anaconda3/envs/fashion/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None t

loss:6.921896 [    0/60000]
loss:2.628282 [ 6400/60000]
loss:2.230845 [12800/60000]
loss:1.963102 [19200/60000]
loss:1.536317 [25600/60000]
loss:1.304635 [32000/60000]
loss:0.947163 [38400/60000]
loss:0.765277 [44800/60000]
Validation Error


/home/percool/anaconda3/envs/fashion/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/percool/anaconda3/envs/fashion/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None t

Accuracy: 11.0%, Avg loss: 0.895487 

Test Error


RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small